# Introduction

The following notebook demonstrates a basic implementation of Latent Dirichlet Allocation to be used as a baseline model. Our goal is to model topics that correctly identify novel discussion topics across 16 distinct government agencies in the Bay Area. 

To demonstrate our workflow quickly, we work with a 1,000-document sample of the full Agenda Watch corpus, which contains more than 20,000 documents of at least 55 words. 

Please note that the notebook contains code adapted from the tutorial ["Evaluate Topic Models: Latent Dirichlet Allocation (LDA)"](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0) by Shashank Kapadia.


# Setup

### Libraries

In [1]:
import pandas as pd
import re
import os
import numpy as np
import pickle 

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy

import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Preprocessing

As a demonstration only, we read in a sample of 1,000 documents.

In [3]:
# Demo: Read in 1,000 example documents sampled from our corpus
df = pd.read_csv('55word_short.csv')
df

### Standardize text

We next perform some basic standardization on the corpus.

In [4]:
# Remove punctuation
df['doc_text'] = df['doc_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert text to lowercase
df['place'] = df['place'].map(lambda x: x.lower())
df['doc_text'] = df['doc_text'].map(lambda x: x.lower())
df['doc_title'] = df['doc_title'].map(lambda x: x.lower())

# Remove dashes from place names and doc titles
df['place'] = df['place'].map(lambda x: x.replace('-', ' '))
df['doc_title'] = df['doc_title'].map(lambda x: x.replace('-', ' '))

### Tokenize

We tokenize the corpus, splitting sentences into into individual words.

In [5]:
# Tokenize

for i in range(len(df)):
  df['doc_text'][i] = gensim.utils.simple_preprocess(str(df['doc_text'][i]), deacc=True)


### Extract bigrams

In [6]:
# Extract just the words
data = df.doc_text.values.tolist()

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

### Remove boilerplate and stop words

We define a custom list of stop words, which includes the names of the government agencies we are scraping as well as common words featured in meeting agendas and minutes.

In [8]:
# Words to remove
place_names = list(pd.unique(df['place']))

doc_titles = list(pd.unique(df['doc_title']))

boilerplate = ['council', 'meeting', 'planning', 'mayor', 'ordinance', 'district',
  'special', 'county', 'city', 'municipal', 'commission', 'committee',
  'amendment', 'agenda', 'minutes', 'district', 'https', 'notice', 'event', 
  'participate', 'report', 'doc_text', 'report', 'advisory', 'name', 'team',
  'printed', 'public', 'public hearing', 'office', 'manager', 'supervisor', 
  'board', 'clerk', 'motion', 'consent', 'calendar', 'finance', 
  'public record', 'code', 'section', 'attorney', 'staff', 'report', 'item', 
  'clerk', 'chair', 'page', 'meet', 'plan', 'call', 'participate', 'ada',
  'civil', 'remotely', 'consider', 'library', 'budget', 'commissions']

We combine this coustom list with a preloaded list of stopwords included in the Gensim package.

In [9]:
# Combine all stop words into one list
standard_stop_words = list(gensim.parsing.preprocessing.STOPWORDS)
custom_stop_words = doc_titles + place_names + boilerplate + standard_stop_words
stop_words = stopwords.words('english')
stop_words.extend(custom_stop_words)

We now define a series of functions we will use in preprocessing.

In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

We apply these functions to our corpus.

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Modeling

We are now ready to prepare the LDA model.

### Make the dictionary and corpus

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

### Build the base model

As an initial pass, we do not tune hyperparameters.

In [13]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=20, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

### Calculate coherence score

In [14]:
# Base model coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48222746300337505


Our coherence score seems a little low, but this is too be expected given that we are using only a subset of our dataset and that we have not yet tuned hyperparameters.

### Tune hyperparameters

In [15]:
# Make a function to compute coherence for different values of:
# corpus, dictionary, k (topics), a (alpha - document-topic density) and b (beta - word-topic density) 
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

We now define a range of possible hyperparameter values for the number of topics, alpha and beta. Since this is only a small sample of documents, we use a more conventional 75%-25% training and test set. With the full corpus, we will likely be able to reserve a smaller proportion of documents as the test set. 

In [ ]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 15
max_topics = 200
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)

100%|██████████| 540/540 [1:09:05<00:00,  6.77s/it]
541it [1:09:12,  6.96s/it]                         
542it [1:09:19,  6.94s/it]
543it [1:09:26,  7.01s/it]
544it [1:09:33,  6.85s/it]
545it [1:09:40,  6.87s/it]
546it [1:09:48,  7.26s/it]
547it [1:09:55,  7.35s/it]
548it [1:10:02,  7.21s/it]
549it [1:10:09,  7.06s/it]
550it [1:10:16,  7.19s/it]
551it [1:10:23,  7.08s/it]
552it [1:10:30,  6.99s/it]
553it [1:10:36,  6.79s/it]
554it [1:10:43,  6.64s/it]
555it [1:10:49,  6.59s/it]
556it [1:10:55,  6.50s/it]
557it [1:11:02,  6.39s/it]
558it [1:11:08,  6.26s/it]
559it [1:11:13,  6.09s/it]
560it [1:11:19,  6.08s/it]
561it [1:11:26,  6.41s/it]
562it [1:11:33,  6.50s/it]
563it [1:11:40,  6.53s/it]
564it [1:11:46,  6.40s/it]
565it [1:11:52,  6.48s/it]
566it [1:12:00,  6.73s/it]
567it [1:12:06,  6.68s/it

# Evaluation

We first assess our results quantitatively and qualitatively.

In [ ]:
#TODO: train the model using the 'best' hyperparameters
#TODO: evaluate coherence
#TODO: preview the actual topics

# Next steps

The work presented here demonstrates some of the promise as well as the shortcomings of using topic modeling on our corpus. Our next steps include the following:

* Conduct further preprocessing. In particular, we will experiment with different ways to remove boilerplate text from our corpus before training a model. When promising avenue is to use a 'diff' strategy to remove identical text from documents produced by the same government agency. Removing boilerplate will decrease the likelihood of forming topics that do not capture the substance of meeting discussions.
* Assess whether topic coherence is the best measure of model performance and consider other possible quantitive metrics to guide model tuning.
* Continue tuning this basic model, adjusting values of hyperparameters within tighter, more-precise windows. 
* Introduce the element of time into this analysis. Can we model different topics every week and then compare each week's topics to previous weeks' topics to detect shifts in the composition of what local governments are discussing?
* Consider which deep learning strategies can be used to improve topic modeling.